In [ ]:
mamba activate cell2loc_prep_r 
#mamba activate cell2location
#pip install jax==0.5
#mamba install ipykernel
#python3 -m ipykernel install --user --name=cell2loc_prep_r --display-name "cell2loc_prep_r"


This next section reesumes Ollie's workflow from Run_cell2location_per_sample.ipynb, from Use scanpy pipeline
(we don't need to run this section as we have processed and QC'd already in R, Ollie does this in scanpy and then goes on to cell2location)
Begin by merging the visium samples to an anndata onject

In [ ]:
#register kernel 
#in terminal, open R
#install.packages('IRkernel')
#IRkernel::installspec(user = TRUE)
#quit()

In [ ]:
library(Seurat)
#library(SeuratDisk)
library(dplyr)
library(HDF5Array)
#library(SpatialExperiment)

samplesheet <- c("SITSA1", "SITSA2", "SITSA3", "SITSA4",
                 "SITSB1", "SITSB2", "SITSB3", "SITSB4", 
                 "SITSC1", "SITSC2", "SITSC3", #"SITSC4", NEED TO GENERATE HDF5 FILES FOR C4 AGAIN
                 "SITSD1", "SITSD2", "SITSD3", "SITSD4",
                 "SITSE2", "SITSE4", "SITSF2", "SITSF4",
                 "SITSG2", "SITSH2")

# Load HVG-filtered RDS files
cluster_files <- list.files(path = "/home/lythgo02/Documents/saved_spe_h5", pattern = "_spe_nnsvg_clusterDE", full.names = TRUE)

names(cluster_files) <- samplesheet

spe_list <- list()
for (sample in names(cluster_files)) {
  spe_list[[sample]] <- loadHDF5SummarizedExperiment(cluster_files[[sample]])
}


In [ ]:

#switch to python env (and switch kernel)

#if not working in notebook
#mamba activate cell2location _cuda118_torch22
#python


In [ ]:

import scanpy as sc
import os


Load each sample’s spatial data from assays.h5 (need to fix SITSC4).
Make gene names unique to avoid duplication problems.
Tag each cell/spot with its sample ID.
Collect  samples in a list (slides) for further processing.

In [ ]:
base_path = ""/mnt/scratchc/fmlab/lythgo02/Spatial/saved_spe_h5ad/" COPIED OVER TO NAS BUT NEED TO COPY TO LOCAL TOO 
samples = [file for file in os.listdir(base_path) 
    if file.endswith("_spe_nnsvg_clusterDE")]  #scans basepath directory and collects folder names matching string 

#result samples = ['SITSA1_spe_nnsvg_clusterDE', 'SITSC3_spe_nnsvg_clusterDE', 'SITSD4_spe_nnsvg_clusterDE']

slides = []

for sample in samples:
    file_path = os.path.join(base_path, sample, "name.h5ad")
    adata = sc.read_h5ad(file_path)
    
    # (Optional) Clean var_names, sync genes, etc.
    adata.var_names_make_unique()    #ensures gene names are unique 
    adata.obs["sample"] = sample     # adds a new column to observation metadata, labels each cell/spot in adata with sample name it came from 
    
    slides.append(adata)